# Panel Regressions: AI Use and Developer Productivity

This notebook estimates fixed-effects panel regressions examining the relationship between AI code assistant usage and developer productivity metrics.

**Input:** `outputs/panel_uq.parquet` (user-quarter panel from `process_data.py`)

**Outputs:** 
- `outputs/main_regressions.pdf` - Main regression coefficient plot
- `outputs/interaction_regressions.pdf` - Experience interaction plot  
- `outputs/placebo_regressions.pdf` - Pre-period placebo plot
- `outputs/ma_extrapolation.pdf` - Measurement error extrapolation plots

In [ ]:
import os
import numpy as np
import pandas as pd
import pyfixest as pf
from marginaleffects import slopes
import matplotlib.pyplot as plt

# Plot styling
plt.rcParams.update({
    "axes.labelsize": 20,
    "axes.titlesize": 20,
    "xtick.labelsize": 15,
    "ytick.labelsize": 15,
    "lines.linewidth": 3,
    "lines.markersize": 8,
    "axes.edgecolor": "black",
    "axes.linewidth": 2,
    "axes.grid": False,
    "figure.figsize": (12, 8),
    "figure.dpi": 300,
    "figure.facecolor": "white"
})

# Paths
DATA_PATH = "outputs/panel_uq.parquet"
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

# MA windows for robustness checks
MA_WINDOWS = [4, 8, 16, 32]

## Load and Prepare Data

In [ ]:
# Load panel data
df = pd.read_parquet(DATA_PATH)
df = df.sort_values(['IDu', 'q']).reset_index(drop=True)

print(f"Panel dimensions: {len(df)} observations, {df['IDu'].nunique()} users")
print(f"Quarter range: {df['q'].min()} to {df['q'].max()}")

In [ ]:
# Create derived variables for regressions

# Bin experience (median split)
df['binned_experience'] = pd.qcut(
    df['experience'], 2, labels=False, duplicates='drop'
)

# Bin lagged AI exposure (quintiles)
df['binned_L_AIavF'] = pd.qcut(
    df['L_AIavF'], 5, labels=False, duplicates='drop'
)

# Create subsets
ma32_subset = df[~df['L_AIma32_iF'].isna()].copy()
df_placebo = df[df['q'] < 20221].copy()  # Pre-ChatGPT period

## Helper Functions

In [ ]:
def extract_ci(model, term):
    """Extract estimate and 95% CI from model tidy output."""
    t = model.tidy()
    return (
        float(t.loc[term, "Estimate"]),
        float(t.loc[term, "2.5%"]),
        float(t.loc[term, "97.5%"])
    )


def run_main_models(data, regressor="L_AIavF"):
    """Run main regression specifications across all outcome variables."""
    dvs = [
        "log_Cuq", "log_Cuq_mfiles", "log_Cuq_wimprt",
        "log_libQC_all", "log_libkQC_all", "log_libLQ_all", "log_libE_all",
        "log_libQC_new_u", "log_libkQC_new_u", "log_libLQ_new_u", "log_libE_new_u"
    ]
    models = []
    for dv in dvs:
        m = pf.feols(
            f"{dv} ~ {regressor} | IDu + q",
            data=data,
            vcov={'CRV1': 'IDu'}
        )
        models.append(m)
    return models, dvs


def run_interaction_models(data):
    """Run models with experience interaction."""
    dvs = [
        "log_Cuq", "log_Cuq_mfiles", "log_Cuq_wimprt",
        "log_libQC_all", "log_libkQC_all", "log_libLQ_all", "log_libE_all",
        "log_libQC_new_u", "log_libkQC_new_u", "log_libLQ_new_u", "log_libE_new_u"
    ]
    models = []
    for dv in dvs:
        m = pf.feols(
            f"{dv} ~ binned_experience + L_AIavF + binned_experience*L_AIavF | IDu + q",
            data=data,
            vcov={'CRV1': 'IDu'}
        )
        models.append(m)
    return models, dvs


def plot_coefficients(models, regressor, filename, title=None):
    """Create coefficient plot with grouped outcomes."""
    # Extract estimates
    ests, los, his = [], [], []
    for m in models:
        est, lo, hi = extract_ci(m, regressor)
        ests.append(est)
        los.append(lo)
        his.append(hi)
    
    ests = np.array(ests)
    yerr = np.vstack([ests - np.array(los), np.array(his) - ests])
    
    # Labels
    labels = [
        'All', 'Multi-file', 'Imports',
        'Combos', 'Combos\n(Top 5k)', 'Combos\n(Groups)', 'Indiv.\nLibs.',
        'Combos', 'Combos\n(Top 5k)', 'Combos\n(Groups)', 'Indiv.\nLibs.'
    ]
    
    # Group spacing
    group_sizes = [3, 4, 4]
    gap = 0.8
    x_spaced = []
    offset = 0.0
    for gsize in group_sizes:
        x_spaced.extend(offset + np.arange(gsize))
        offset += gsize + gap
    x_spaced = np.array(x_spaced)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12.0, 5.0), dpi=300)
    ax.errorbar(x_spaced, ests, yerr=yerr, fmt="o", capsize=3, color='black')
    ax.axhline(0, lw=1, color="black")
    ax.set_xticks(x_spaced)
    ax.set_xticklabels(labels, rotation=0, ha="center", fontsize=13)
    ax.set_xlim(x_spaced[0] - 0.6, x_spaced[-1] + 0.6)
    ax.set_ylabel("Estimated marginal\neffect of AI use")
    ax.tick_params(axis="y", labelsize=13)
    
    # Group labels
    sec = ax.secondary_xaxis(location=-0.17)
    sec.spines['bottom'].set_visible(False)
    g1_center = x_spaced[0:3].mean()
    g2_center = x_spaced[3:7].mean()
    g3_center = x_spaced[7:11].mean()
    sec.set_xticks(
        [g1_center, g2_center, g3_center],
        labels=['Commits\n(log)', 'Library Use\n(log)', 'Library Entry\n(log)'],
        size=20
    )
    sec.tick_params('x', length=0)
    
    if title:
        plt.title(title)
    
    fig.tight_layout()
    plt.savefig(f'{OUTDIR}/{filename}')
    plt.show()

## Main Results

In [ ]:
# Run main regressions
models, dvs = run_main_models(df)

# Display regression table
pf.etable(
    models,
    head_order="hd",
    model_heads=[
        'Commits', 'Commits', 'Commits',
        'Library Use', 'Library Use', 'Library Use', 'Library Use',
        'Library Entry', 'Library Entry', 'Library Entry', 'Library Entry'
    ],
    labels={
        'log_Cuq': 'All Commits (log)',
        'log_Cuq_mfiles': 'Multi-file (log)',
        'log_Cuq_wimprt': 'Imports (log)',
        'log_libQC_all': 'Combos (log)',
        'log_libkQC_all': 'Combos (5k) (log)',
        'log_libLQ_all': 'Combos (groups) (log)',
        'log_libE_all': 'Individual Libs. (log)',
        'log_libQC_new_u': 'Combos (log)',
        'log_libkQC_new_u': 'Combos (5k) (log)',
        'log_libLQ_new_u': 'Combos (groups) (log)',
        'log_libE_new_u': 'Individual Libs. (log)',
        'q': 'Quarter', 'IDu': 'User', 'L_AIavF': 'AI Use'
    },
    notes="Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001."
)

In [ ]:
# Coefficient plot
plot_coefficients(models, 'L_AIavF', 'main_regressions.pdf')

In [ ]:
# LaTeX table output
print(
    pf.etable(
        models,
        head_order="hd",
        model_heads=[
            'Commits', 'Commits', 'Commits',
            'Library Use', 'Library Use', 'Library Use', 'Library Use',
            'Library Entry', 'Library Entry', 'Library Entry', 'Library Entry'
        ],
        labels={
            'log_Cuq': 'All Commits (log)',
            'log_Cuq_mfiles': 'Multi-file (log)',
            'log_Cuq_wimprt': 'Imports (log)',
            'log_libQC_all': 'Combos (log)',
            'log_libkQC_all': 'Combos (5k) (log)',
            'log_libLQ_all': 'Combos (groups) (log)',
            'log_libE_all': 'Individual Libs. (log)',
            'log_libQC_new_u': 'Combos (log)',
            'log_libkQC_new_u': 'Combos (5k) (log)',
            'log_libLQ_new_u': 'Combos (groups) (log)',
            'log_libE_new_u': 'Individual Libs. (log)',
            'q': 'Quarter', 'IDu': 'User', 'L_AIavF': 'AI Use'
        },
        notes="Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001.",
        type="tex"
    )
)

## Experience Interaction Models

In [ ]:
# Run interaction models
int_models, dvs = run_interaction_models(df)

# Display table
pf.etable(
    int_models,
    head_order="hd",
    model_heads=[
        'Commits', 'Commits', 'Commits',
        'Library Use', 'Library Use', 'Library Use', 'Library Use',
        'Library Entry', 'Library Entry', 'Library Entry', 'Library Entry'
    ],
    labels={
        'q': 'Quarter', 'IDu': 'User', 'L_AIavF': 'AI Use'
    },
    notes="Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001."
)

In [ ]:
# LaTeX output for interaction models
print(
    pf.etable(
        int_models,
        head_order="hd",
        model_heads=[
            'Commits', 'Commits', 'Commits',
            'Library Use', 'Library Use', 'Library Use', 'Library Use',
            'Library Entry', 'Library Entry', 'Library Entry', 'Library Entry'
        ],
        labels={'q': 'Quarter', 'IDu': 'User', 'L_AIavF': 'AI Use'},
        notes="Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001.",
        type="tex"
    )
)

In [ ]:
# Plot interaction effects by experience group
def plot_interaction_effects(models):
    """Plot AI effect separately for low/high experience groups."""
    labels = [
        'All', 'Multi-file', 'Imports',
        'Combos', 'Combos\n(Top 5k)', 'Combos\n(Groups)', 'Indiv.\nLibs.',
        'Combos', 'Combos\n(Top 5k)', 'Combos\n(Groups)', 'Indiv.\nLibs.'
    ]
    
    # Extract slopes by experience group
    low_ests, low_los, low_his = [], [], []
    high_ests, high_los, high_his = [], [], []
    
    for m in models:
        sl = slopes(m, variables='L_AIavF', by='binned_experience', vcov=True).to_pandas()
        sl['binned_experience'] = sl['binned_experience'].astype(int)
        sl = sl.sort_values('binned_experience')
        
        low = sl[sl['binned_experience'] == 0].iloc[0]
        high = sl[sl['binned_experience'] == 1].iloc[0]
        
        low_ests.append(low['estimate'])
        low_los.append(low['conf_low'])
        low_his.append(low['conf_high'])
        high_ests.append(high['estimate'])
        high_los.append(high['conf_low'])
        high_his.append(high['conf_high'])
    
    low_ests = np.array(low_ests)
    high_ests = np.array(high_ests)
    low_yerr = np.vstack([low_ests - np.array(low_los), np.array(low_his) - low_ests])
    high_yerr = np.vstack([high_ests - np.array(high_los), np.array(high_his) - high_ests])
    
    # Spacing
    group_sizes = [3, 4, 4]
    gap = 0.8
    x_spaced = []
    offset = 0.0
    for gsize in group_sizes:
        x_spaced.extend(offset + np.arange(gsize))
        offset += gsize + gap
    x_spaced = np.array(x_spaced)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12.0, 5.0), dpi=300)
    width = 0.15
    
    ax.errorbar(x_spaced - width, low_ests, yerr=low_yerr, fmt="o", capsize=3, 
                color='#1f77b4', label='Early-Career')
    ax.errorbar(x_spaced + width, high_ests, yerr=high_yerr, fmt="o", capsize=3,
                color='#ff7f0e', label='Senior-Level')
    
    ax.axhline(0, lw=1, color="black")
    ax.set_xticks(x_spaced)
    ax.set_xticklabels(labels, rotation=0, ha="center", fontsize=13)
    ax.set_xlim(x_spaced[0] - 0.6, x_spaced[-1] + 0.6)
    ax.set_ylabel("Estimated marginal\neffect of AI use")
    ax.legend(frameon=False, fontsize=13)
    ax.tick_params(axis="y", labelsize=13)
    
    # Group labels
    sec = ax.secondary_xaxis(location=-0.17)
    sec.spines['bottom'].set_visible(False)
    sec.set_xticks(
        [x_spaced[0:3].mean(), x_spaced[3:7].mean(), x_spaced[7:11].mean()],
        labels=['Commits\n(log)', 'Library Use\n(log)', 'Library Entry\n(log)'],
        size=20
    )
    sec.tick_params('x', length=0)
    
    fig.tight_layout()
    plt.savefig(f'{OUTDIR}/interaction_regressions.pdf')
    plt.show()

plot_interaction_effects(int_models)

## Nonlinearity Check (Quintile Dummies)

In [ ]:
# Run nonlinearity models with quintile dummies
dvs = [
    "log_Cuq", "log_Cuq_mfiles", "log_Cuq_wimprt",
    "log_libQC_all", "log_libkQC_all", "log_libLQ_all", "log_libE_all",
    "log_libQC_new_u", "log_libkQC_new_u", "log_libLQ_new_u", "log_libE_new_u"
]

nonlin_models = []
for dv in dvs:
    m = pf.feols(
        f"{dv} ~ C(binned_L_AIavF) | IDu + q",
        data=df,
        vcov={'CRV1': 'IDu'}
    )
    nonlin_models.append(m)

pf.etable(nonlin_models, coef_fmt="b \n (se) \n [p]")

## Placebo Test (Pre-ChatGPT Period)

In [ ]:
# Run placebo regressions on pre-2022Q1 data
placebo_models, _ = run_main_models(df_placebo)

# Display table
print(
    pf.etable(
        placebo_models,
        head_order="hd",
        model_heads=[
            'Commits', 'Commits', 'Commits',
            'Library Use', 'Library Use', 'Library Use', 'Library Use',
            'Library Entry', 'Library Entry', 'Library Entry', 'Library Entry'
        ],
        labels={'q': 'Quarter', 'IDu': 'User', 'L_AIavF': 'AI Use'},
        notes="Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001.",
        type="tex"
    )
)

In [ ]:
# Placebo coefficient plot
plot_coefficients(
    placebo_models, 'L_AIavF', 'placebo_regressions.pdf',
    title='Placebo Regression (Pre-ChatGPT)'
)

## Measurement Error: MA Window Extrapolation

Test robustness of estimates across different moving average windows for AI exposure.
Longer windows reduce measurement error; extrapolating to infinite window (1/w → 0)
gives the measurement-error-corrected estimate.

In [ ]:
def ma_col(w):
    """Get MA column name for window w."""
    return f"L_AIma{w}_iF"


def compute_ma_series(data, dv):
    """Compute estimates across MA windows for extrapolation."""
    xs, ests, los, his = [], [], [], []
    for w in MA_WINDOWS:
        reg = ma_col(w)
        m = pf.feols(f"{dv} ~ {reg} | IDu + q", data=data, vcov={'CRV1': 'IDu'})
        est, lo, hi = extract_ci(m, reg)
        xs.append(1.0 / w)
        ests.append(est)
        los.append(lo)
        his.append(hi)
    return np.array(xs), np.array(ests), np.array(los), np.array(his)


def plot_ma_extrapolation():
    """Create MA extrapolation plots for all DVs."""
    dvs = [
        "log_Cuq", "log_Cuq_mfiles", "log_Cuq_wimprt",
        "log_libQC_all", "log_libkQC_all", "log_libLQ_all", "log_libE_all",
        "log_libQC_new_u", "log_libkQC_new_u", "log_libLQ_new_u", "log_libE_new_u"
    ]
    titles = [
        'All Commits', 'Multi-file', 'Imports',
        'Combos', 'Combos (Top 5k)', 'Combos (Groups)', 'Indiv. Libs.',
        'Combos (New)', 'Combos 5k (New)', 'Combos Groups (New)', 'Indiv. Libs. (New)'
    ]
    
    xticks = [1/32, 1/16, 1/8, 1/4]
    xticklabels = [r"$\frac{1}{32}$", r"$\frac{1}{16}$", r"$\frac{1}{8}$", r"$\frac{1}{4}$"]
    
    fig, axes = plt.subplots(4, 3, figsize=(16, 12), dpi=300)
    axes = axes.flatten()
    
    for i, (dv, title) in enumerate(zip(dvs, titles)):
        if i >= len(axes):
            break
        ax = axes[i]
        xs, ests, los, his = compute_ma_series(df, dv)
        
        # Fit extrapolation line
        coeffs = np.polyfit(xs, ests, deg=1)
        a, b = coeffs[1], coeffs[0]
        x_line = np.linspace(0, 0.3, 200)
        y_line = a + b * x_line
        
        # Plot
        yerr = np.vstack([ests - los, his - ests])
        ax.errorbar(xs, ests, yerr=yerr, fmt="o", capsize=3)
        ax.plot(x_line, y_line, linewidth=1)
        ax.axhline(0, linewidth=0.8, linestyle="--", alpha=0.6)
        
        ax.set_xlim(0, 0.3)
        ax.set_xticks(xticks)
        ax.set_xticklabels(xticklabels)
        ax.set_title(title, fontsize=12)
        ax.text(
            0.93, 0.93, f"Slope: {b:.3g}",
            transform=ax.transAxes, ha="right", va="top",
            fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
        )
    
    # Hide unused subplot
    axes[-1].axis('off')
    
    fig.supxlabel('1 / MA Window Size', fontsize=14)
    fig.supylabel('Estimated Coefficient', fontsize=14)
    fig.tight_layout()
    plt.savefig(f'{OUTDIR}/ma_extrapolation.pdf')
    plt.show()

plot_ma_extrapolation()